In [1]:
import glob 
import pandas as pd
import os 
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "colab"

In [2]:
def _read_file(file, folder):
    file_path = os.path.join(folder, f"{file}.csv")
    if not os.path.exists(file_path):
        print(f"Error: File untuk {file} tidak ditemukan di folder tersebut.")
        return None
        
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date', ascending=True)   

    return df

In [3]:
def _data_processing(df):
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA100'] = df['Close'].rolling(window=100).mean()

    max_date = df['Date'].max()
    six_months_ago = max_date - pd.DateOffset(months=6)
    df_plot = df[df['Date'] >= six_months_ago].copy()

    return df_plot

In [10]:
def _visualize(df_plot, file_name):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x = df_plot['Date'],
        open = df_plot['Open'],
        high = df_plot['High'],
        low = df_plot['Low'],
        close = df_plot['Close'],
        name = 'Candlestick'
    ))

    fig.add_trace(go.Scatter(x = df_plot['Date'], y = df_plot['MA20'], 
                             line = dict(color = 'blue', width = 1.5), name = 'MA 20'))
    
    fig.add_trace(go.Scatter(x = df_plot['Date'], y = df_plot['MA50'], 
                             line = dict(color = 'orange', width = 1.5), name = 'MA 50'))
    
    fig.add_trace(go.Scatter(x = df_plot['Date'], y = df_plot['MA100'], 
                             line = dict(color = 'red', width = 1.5), name = 'MA 100'))

    fig.update_layout(
        title = f'Analisis Teknikal Saham {file_name} (6 Bulan Terakhir)',
        yaxis_title = 'Harga (IDR)',
        xaxis_title = 'Tanggal',
        xaxis_rangeslider_visible = False, 
        template = 'plotly_white',
        legend = dict(orientation = "h", yanchor = "bottom", y = 1.02, xanchor = "right", x=1)
    )

    return fig

In [11]:
def stocks_visualization(file, folder):
    df = _read_file(file, folder)
    if df is None:
        return

    df_plot = _data_processing(df)

    fig = _visualize(df_plot, file)
    
    return fig

In [12]:
stocks_visualization('BBCA', 'historical')